In [55]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

# !pip install torch
import torch
import tarfile
import torch.nn as nn
import shutil
from torch import optim
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

!wget http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz
  
file = tarfile.open('Question_Answer_Dataset_v1.2.tar.gz')  # open file
file.extractall('./data') # extract files  
file.close()# close file

file_path = './data/question_answer_pairs.txt'
shutil.move("./data/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt", file_path)

--2023-04-27 11:37:33--  http://www.cs.cmu.edu/~ark/QA-data/data/Question_Answer_Dataset_v1.2.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8254496 (7.9M) [application/x-gzip]
Saving to: ‘Question_Answer_Dataset_v1.2.tar.gz.6’

Question_Answer_Dat 100%[===================>]   7.87M  4.31MB/s    in 1.8s    

2023-04-27 11:37:35 (4.31 MB/s) - ‘Question_Answer_Dataset_v1.2.tar.gz.6’ saved [8254496/8254496]



'./data/question_answer_pairs.txt'

In [56]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

class Data:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readData(file_path):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('{}'.format(file_path), encoding='latin-1').read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[1:3]] for l in lines]
    pairs = pairs[1:]

    question_details = Data('question')  # CHECK
    answer_details = Data('answer')  # CHECK

    return question_details, answer_details, pairs


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH 

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(file_path):
    question_details, answer_details, pairs = readData(file_path)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        question_details.addSentence(pair[0])
        answer_details.addSentence(pair[1])
    print("Counted words:")
    print(question_details.name, question_details.n_words)
    print(answer_details.name, answer_details.n_words)
    return question_details, answer_details, pairs


question_details, answer_details, pairs = prepareData(file_path)
print(random.choice(pairs))

Reading lines...
Read 1715 sentence pairs
Trimmed to 926 sentence pairs
Counting words...
Counted words:
question 992
answer 670
['have kangaroos fared well since european settlement ?', 'yes']


In [57]:
# (pairs)  # remove ? and . from end. After that, remove whitestapces from end

answer_details.name, answer_details.n_words, question_details.name, question_details.n_words
# (answer_details.word2index, answer_details.index2word, answer_details.word2count)

('answer', 670, 'question', 992)

In [58]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(question_details, pair[0])
    target_tensor = tensorFromSentence(answer_details, pair[1])
    return (input_tensor, target_tensor)


import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [59]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)  # this is SOS token
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output, hidden = self.gru(output, hidden)
        output = F.relu(output)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [60]:
!pip install bert_score==0.3.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
teacher_forcing_ratio = 0.25
import math
from bert_score import score

def unigram_precision(predicted_tensor, target_tensor):
    predicted_tensor_flatten = torch.flatten(predicted_tensor)
    predicted_tensor_frequencies = torch.bincount(predicted_tensor_flatten.type(torch.int64))
    total_predicted_tensor_frequencies = torch.sum(predicted_tensor_frequencies)  # Denominator for unigram precision computation.

    target_tensor_flatten = torch.flatten(target_tensor)
    target_tensor_frequencies = torch.bincount(target_tensor_flatten.type(torch.int64))
    # print(predicted_tensor_flatten, predicted_tensor_frequencies)  
    # print(target_tensor_flatten, target_tensor_frequencies)
    
    len_pred = list(predicted_tensor_frequencies.shape)[0]
    len_target = list(target_tensor_frequencies.shape)[0]
    # make_len_equal_target_pred_by_appending_zeros
    if len_pred > len_target:  # len of pred is more than len of actual
        target_tensor_frequencies = nn.ConstantPad1d((0, len_pred - len_target), 0)(target_tensor_frequencies)
    elif len_pred < len_target:
        predicted_tensor_frequencies = nn.ConstantPad1d((0, len_target - len_pred), 0)(predicted_tensor_frequencies)
    else:
        pass

    min_pred_target_frequencies = torch.min(predicted_tensor_frequencies, target_tensor_frequencies)  # Clip count - keep min of pred and actual freq
    # print(predicted_tensor_frequencies, target_tensor_frequencies, min_pred_target_frequencies, torch.sum(min_pred_target_frequencies), torch.sum(min_pred_target_frequencies).item())
    total_min_pred_target_tensor_frequencies = torch.sum(min_pred_target_frequencies)  # Numerator for unigram precision computation.
    # print('sid', total_min_pred_target_tensor_frequencies.item(), total_predicted_tensor_frequencies.item())
    try:
        unigram_precision_val = total_min_pred_target_tensor_frequencies.item() / total_predicted_tensor_frequencies.item()
    except ZeroDivisionError:
        unigram_precision_val = 0
    # print('u prc val :', unigram_precision_val)
    return unigram_precision_val

def brevity_panelty(predicted_target_trimmed, target_tensor_trimmed):  # penalize machine output which is of lesser length than actual output
    count_elements_in_predicted_target_trimmed = predicted_target_trimmed.numel()
    count_elements_in_target_tensor_trimmed = target_tensor_trimmed.numel()
    if int(count_elements_in_predicted_target_trimmed) < int(count_elements_in_target_tensor_trimmed):
        # print(count_elements_in_target_tensor_trimmed, count_elements_in_predicted_target_trimmed)
        try:
            penalty = math.exp((1-(count_elements_in_target_tensor_trimmed/count_elements_in_predicted_target_trimmed)))
        except ZeroDivisionError:
            penalty = math.exp(1)
        return penalty
    else:
        return 1

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    prob_for_perplexity = []

    if True:  # use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        predicted_target = torch.zeros(max_length, 1, device=device)  # evaluation
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            
            prob_for_perplexity.append(topv.squeeze())

            decoder_input_for_precision = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            predicted_target[di] = decoder_input_for_precision  # evaluation
            decoder_input = target_tensor[di]  # Teacher forcing

        perplexity = math.exp((-1/target_length) * sum(prob_for_perplexity))  # Compute Perplexity

        predicted_target_trimmed = predicted_target[predicted_target.sum(dim=1) != 0]  # remove unwanted rows at the end with 0 value.
        predicted_target_trimmed = predicted_target_trimmed[predicted_target_trimmed[:, 0] !=1]  # remove last row in case it has EOS token
        row_exclude = target_tensor.shape[0]-1
        target_tensor_trimmed = torch.cat((target_tensor[:row_exclude],target_tensor[row_exclude+1:]))  # remove EOS token from end

        # Compute Bert Score between Predicted and Target Sent
        # print("siddharth ", predicted_target_trimmed.shape, type(predicted_target_trimmed), predicted_target_trimmed)
        decoded_words = []  # Predicted Sentence
        decoded_sentence_list = []
        for predicted_target_index in (predicted_target_trimmed.squeeze()).type('torch.IntTensor'):
            decoded_words.append(answer_details.index2word[predicted_target_index.item()])
        decoded_sentence = ' '.join(decoded_words)
        decoded_sentence_list.append(decoded_sentence)
        target_words = []
        target_sentence_list = []
        for target_tensor in (target_tensor_trimmed.squeeze()).type('torch.IntTensor'):
            target_words.append(answer_details.index2word[target_tensor.item()])
        target_sentence = ' '.join(target_words)
        target_sentence_list.append(target_sentence)
        BERT_P, BERT_R, BERT_F1 = score(decoded_sentence_list, target_sentence_list, lang="en", verbose=True)

        # Calculate unigram precision for predicted and target. Continue to compute penalty and BLEU score.
        unigram_precision_value = unigram_precision(predicted_target_trimmed, target_tensor_trimmed)
        modified_n_gram_precision = math.sqrt(unigram_precision_value)  # n is 2 - square root - since we are using unigram and bigrams.
        brevity_panelty_val = brevity_panelty(predicted_target_trimmed, target_tensor_trimmed)
        BLEU = brevity_panelty_val * modified_n_gram_precision

        # intersection of pred and target to get correct prediction count
        indices = torch.zeros_like(target_tensor_trimmed, dtype = torch.uint8, device = 'cuda')
        for elem in predicted_target_trimmed:
            indices = indices | (target_tensor_trimmed == elem)  
        intersection = target_tensor_trimmed[indices]  
        indices_backward = torch.zeros_like(predicted_target_trimmed, dtype = torch.uint8, device = 'cuda')
        for elem in target_tensor_trimmed:
            indices_backward = indices_backward | (predicted_target_trimmed == elem)  
        intersection_backward = predicted_target_trimmed[indices_backward]

        correct_predictions_count = min(intersection.shape[0], intersection_backward.shape[0])

        # Generic Precision value and Recall value
        total_predictions_count = predicted_target_trimmed.shape[0]
        if total_predictions_count != 0:
            precision = correct_predictions_count / total_predictions_count
        else:
            precision = 0
        total_actual_count = target_tensor_trimmed.shape[0]
        recall = correct_predictions_count / total_actual_count

    else:
        # Without teacher forcing: use its own predictions as the next input
        predicted_target = torch.zeros(max_length, 1, device=device)  # evaluation
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)

            prob_for_perplexity.append(topv.squeeze())

            decoder_input = topi.squeeze().detach()  # detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            predicted_target[di] = decoder_input  # evaluation
            if decoder_input.item() == EOS_token:
                break

        perplexity = math.exp((-1/target_length) * sum(prob_for_perplexity))

        predicted_target_trimmed = predicted_target[predicted_target.sum(dim=1) != 0]  # remove unwanted rows at the end with 0 value.
        predicted_target_trimmed = predicted_target_trimmed[predicted_target_trimmed[:, 0] !=1]  # remove last row in case it has EOS token
        row_exclude = target_tensor.shape[0]-1
        target_tensor_trimmed = torch.cat((target_tensor[:row_exclude],target_tensor[row_exclude+1:]))  # remove EOS token from end
        # print('pred and target ', predicted_target_trimmed , target_tensor_trimmed)
        # Calculate unigram precision for predicted and target. Continue to compute penalty and BLEU score.
        unigram_precision_value = unigram_precision(predicted_target_trimmed, target_tensor_trimmed)
        modified_n_gram_precision = math.sqrt(unigram_precision_value)  # n is 2 - square root - since we are using unigram and bigrams.
        brevity_panelty_val = brevity_panelty(predicted_target_trimmed, target_tensor_trimmed)
        # print(brevity_panelty_val, modified_n_gram_precision)
        BLEU = brevity_panelty_val * modified_n_gram_precision
        # print(unigram_precision_value, modified_n_gram_precision)
        # intersection of pred and target to get correct prediction count
        indices = torch.zeros_like(target_tensor_trimmed, dtype = torch.uint8, device = 'cuda')
        for elem in predicted_target_trimmed:
            indices = indices | (target_tensor_trimmed == elem)  
        intersection = target_tensor_trimmed[indices]  
        indices_backward = torch.zeros_like(predicted_target_trimmed, dtype = torch.uint8, device = 'cuda')
        for elem in target_tensor_trimmed:
            indices_backward = indices_backward | (predicted_target_trimmed == elem)  
        intersection_backward = predicted_target_trimmed[indices_backward]

        correct_predictions_count = min(intersection.shape[0], intersection_backward.shape[0])

        # Generic Precision value and Recall value
        total_predictions_count = predicted_target_trimmed.shape[0]
        if total_predictions_count != 0:
            precision = correct_predictions_count / total_predictions_count
        else:
            precision = 0

        total_actual_count = target_tensor_trimmed.shape[0]
        recall = correct_predictions_count / total_actual_count

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length, precision, recall, BLEU, perplexity

In [62]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print_precision_total = 0  # Reset every plot_every
    print_recall_total = 0  # Reset every plot_every
    print_BLEU_total = 0  # Reset every plot_every
    print_perplexity_total = 0 # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss, precision, recall, BLEU, perplexity = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        print_precision_total += precision
        print_recall_total += recall
        print_BLEU_total += BLEU
        print_perplexity_total += perplexity

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_precision_avg = print_precision_total / print_every
            print_recall_avg = print_recall_total / print_every
            print_BLEU_avg = print_BLEU_total / print_every
            print_perplexity_avg = print_perplexity_total / print_every
            print_loss_total = 0
            print_precision_total = 0
            print_recall_total = 0
            print_BLEU_total = 0
            print_perplexity_total = 0
            print('%s (%d %d%%) %.4f  Precision: %.4f  Recall: %.4f  F1: %.4f, BLEU: %.4f, Perplexity: %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg, print_precision_avg, print_recall_avg,
                                         ((2*print_precision_avg*print_recall_avg)/(print_recall_avg+print_precision_avg)), print_BLEU_avg, print_perplexity_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [63]:
hidden_size = 256
encoder1 = EncoderRNN(question_details.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, answer_details.n_words, dropout_p=0.1).to(device)

# Order of Performance Matrics - Precision, Recall, F1, BLEU, Perplexity
# trainIters(encoder1, attn_decoder1, 55000, print_every=5000)
trainIters(encoder1, attn_decoder1, 50, print_every=5)   

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

AttributeError: ignored

In [ ]:
# Precision - How many correct predictions were made out of total predictions made? - We check the number of words in prediction that are present in 
#  reference and divide it by total number of words in prediction. 
# Recall - How many correct predictions were made out of the total actuals? - We check the number of words in prediction that are present in 
#  reference and divide it by total number of words in actual.
# F1 score - 2*precision*recall / (precision + recall)
# BLEU Score  - Refer BLUE score PPT from Mtech NLP Lecture
# Perplexity - Exponential of negative log likelihood.

In [ ]:
# def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
#     with torch.no_grad():
#         input_tensor = tensorFromSentence(question_details, sentence)
#         input_length = input_tensor.size()[0]
#         encoder_hidden = encoder.initHidden()

#         encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

#         for ei in range(input_length):
#             encoder_output, encoder_hidden = encoder(input_tensor[ei],
#                                                      encoder_hidden)
#             encoder_outputs[ei] += encoder_output[0, 0]

#         decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

#         decoder_hidden = encoder_hidden

#         decoded_words = []
#         decoder_attentions = torch.zeros(max_length, max_length)

#         for di in range(max_length):
#             decoder_output, decoder_hidden, decoder_attention = decoder(
#                 decoder_input, decoder_hidden, encoder_outputs)
#             decoder_attentions[di] = decoder_attention.data
#             topv, topi = decoder_output.data.topk(1)
#             if topi.item() == EOS_token:
#                 decoded_words.append('<EOS>')
#                 break
#             else:
#                 decoded_words.append(answer_details.index2word[topi.item()])

#             decoder_input = topi.squeeze().detach()

#         return decoded_words, decoder_attentions[:di + 1]

# def evaluateRandomly(encoder, decoder, n=10):
#     for i in range(n):
#         pair = random.choice(pairs)
#         print('>', pair[0])
#         print('=', pair[1])
#         output_words, attentions = evaluate(encoder, decoder, pair[0])
#         output_sentence = ' '.join(output_words)
#         print('<', output_sentence)
#         print('')

# evaluateRandomly(encoder1, attn_decoder1)